In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from matplotlib import pyplot as plt
import scipy.io
import scipy.ndimage as ndi
from PIL import Image

Check GPU

In [2]:
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
solve_cudnn_error()

1 Physical GPUs, 1 Logical GPUs


Training Data Preprocess

In [4]:
#Resize Image to (50,130)
mat = scipy.io.loadmat('train.mat')
x = mat['x']
height = 50
width = 130
resize = []
for i in range(x.size):
    factor_h = height/x.item(i).shape[0]
    factor_w = width/x.item(i).shape[1]
    resize.append(ndi.zoom(x.item(i), (factor_h,factor_w,1)))

In [5]:
for i in range(5):
    img = Image.fromarray(resize[i], 'RGB')
    file_name = "my"
    file_name = file_name + str(i) + ".png"
    img.save(file_name)
    img.show()
    print(resize[i].shape)
x_train = np.array(resize)
print(x_train.shape)

(50, 130, 3)
(50, 130, 3)
(50, 130, 3)
(50, 130, 3)
(50, 130, 3)
(5000, 50, 130, 3)


In [6]:
#(4,5000,19)    4 digit, 5000 data, 19 class 
y = mat['y']
onehot = mat['y_onehot']
y_train = []
lists = [ [] for _ in range(4) ]
for i in range(y.size):
    if i%4==0:
        lists[0].append(onehot[i])
    elif i%4==1:
        lists[1].append(onehot[i])
    elif i%4==2:
        lists[2].append(onehot[i])
    else: 
        lists[3].append(onehot[i])
for i in range(4):
    lists[i] = np.array(lists[i])
    y_train.append(lists[i])

Model Define and Training

In [7]:
def four_digit_model(input_shape):
    X_input = Input(shape = input_shape)
    out = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(X_input)
    out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(0.3)(out)
    out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(out)
    out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(0.3)(out)
    out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(out)
    out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(0.3)(out)
    out = Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Flatten()(out)
    out = Dropout(0.3)(out)
    out = [Dense(19, name='digit1', activation='softmax')(out),\
        Dense(19, name='digit2', activation='softmax')(out),\
        Dense(19, name='digit3', activation='softmax')(out),\
        Dense(19, name='digit4', activation='softmax')(out)]
    model = Model(inputs = X_input, outputs = out, name='four_digit_Model')  
    return model
md = four_digit_model((height,width,3))
md.summary()
md.compile(optimizer = "adam", loss = "categorical_crossentropy",  metrics = [tf.keras.metrics.CategoricalAccuracy(name="accuracy")])
md.fit(x_train,y_train, batch_size=16, epochs=20, validation_split = 0.1, shuffle = True)
md.save('four_digit_model.h5')

Model: "four_digit_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50, 130, 3)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 50, 130, 32)  896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 48, 128, 32)  9248        conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 48, 128, 32)  128         conv2d_1[0][0]                   
___________________________________________________________________________________

Test Data Preprocess

In [8]:
mat = scipy.io.loadmat('test.mat')
x = mat['x']
height = 50
width = 130
resize = []
for i in range(x.size):
    factor_h = height/x.item(i).shape[0]
    factor_w = width/x.item(i).shape[1]
    resize.append(ndi.zoom(x.item(i), (factor_h,factor_w,1)))
x_test = np.array(resize)
print(x_test.shape)

(3000, 50, 130, 3)


Label Preprocess

In [9]:
y = mat['y']
onehot = mat['y_onehot']
y_test = []
lists = [ [] for _ in range(4) ]
for i in range(y.size):
    if i%4==0:
        lists[0].append(onehot[i])
    elif i%4==1:
        lists[1].append(onehot[i])
    elif i%4==2:
        lists[2].append(onehot[i])
    else: 
        lists[3].append(onehot[i])
for i in range(4):
    lists[i] = np.array(lists[i])
    y_test.append(lists[i])
print(y_test)

[array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint8), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)]


In [10]:
y = mat['y']
code = {}
value = ['2','3','4','5','7','9','A','C','F','H','K','M','N','P','Q','R','T','Y','Z']
for i in range(19):
    code[i] = value[i]
y_label = []
i = 0
while i < y.size:
    label=""
    for j in range(4):
        label+=code[y.item(i)]
        i+=1   
    y_label.append(label)
print(len(y_label))

3000


Model Evaluation

In [11]:
md = load_model('four_digit_model.h5')
score =  md.evaluate(x_test, y_test, batch_size= 16)
print(score)
predict = [np.argmax(md.predict(x_test)[i],axis = 1) for i in range(4)]

188/188 [==============================] - 2s 7ms/step - loss: 0.8231 - digit1_loss: 0.1750 - digit2_loss: 0.2231 - digit3_loss: 0.1471 - digit4_loss: 0.2779 - digit1_digit1_accuracy: 0.9900 - digit2_digit2_accuracy: 0.9853 - digit3_digit3_accuracy: 0.9920 - digit4_digit4_accuracy: 0.9887
[0.8231255412101746, 0.17501236498355865, 0.22310516238212585, 0.14709210395812988, 0.27791619300842285, 0.9900000095367432, 0.9853333234786987, 0.9919999837875366, 0.9886666536331177]


Performance

In [12]:
one_w = 0
four_w = 0
for i in range(3000):
    y_pred=""
    wrong = 0
    for j in range(4):
        y_pred+=code[predict[j][i]]
        if y_pred[j]!=y_label[i][j] :
            if wrong==0:
                four_w+=1
            wrong+=1
            one_w+=1
    if(i<40):
        print("predict = ['"+str(y_pred[0])+"','"+str(y_pred[1])+"','"+str(y_pred[2])+"','"+str(y_pred[3])+"']")
        print(" answer = ['"+str(y_label[i][0])+"','"+str(y_label[i][1])+"','"+str(y_label[i][2])+"','"+str(y_label[i][3])+"'] ,wrong words = "+str(wrong))
print("\n\n3000 picture total wrong = "+str(four_w))
print(" one word accuracy : "+str((12000-one_w)/12000))
print("four word accuracy : "+str((3000-four_w)/3000))

predict = ['7','F','9','4']
 answer = ['7','F','9','4'] ,wrong words = 0
predict = ['4','2','T','7']
 answer = ['4','2','T','7'] ,wrong words = 0
predict = ['C','Y','T','F']
 answer = ['C','Y','T','F'] ,wrong words = 0
predict = ['T','M','F','T']
 answer = ['T','M','F','T'] ,wrong words = 0
predict = ['P','7','K','R']
 answer = ['P','7','K','R'] ,wrong words = 0
predict = ['N','2','Z','K']
 answer = ['N','2','Z','K'] ,wrong words = 0
predict = ['7','4','F','Y']
 answer = ['7','4','F','Y'] ,wrong words = 0
predict = ['Z','Z','5','C']
 answer = ['Z','Z','5','C'] ,wrong words = 0
predict = ['7','Z','4','4']
 answer = ['7','Z','4','4'] ,wrong words = 0
predict = ['R','4','Z','C']
 answer = ['R','4','Z','C'] ,wrong words = 0
predict = ['Y','2','5','T']
 answer = ['Y','2','5','T'] ,wrong words = 0
predict = ['F','M','2','2']
 answer = ['F','M','2','2'] ,wrong words = 0
predict = ['Y','4','M','4']
 answer = ['Y','4','M','4'] ,wrong words = 0
predict = ['M','2','Q','2']
 answer = ['M','2','Q',